In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "9BtCOyQ3kChJ"
   },
   "outputs": [],
   "source": [
    "# Install Java, Spark, and Findspark\n",
    "!apt-get install openjdk-8-jdk-headless -qq > /dev/null\n",
    "!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz\n",
    "!tar xf spark-2.4.5-bin-hadoop2.7.tgz\n",
    "!pip install -q findspark\n",
    "\n",
    "# Set Environment Variables\n",
    "import os\n",
    "os.environ[\"JAVA_HOME\"] = \"/usr/lib/jvm/java-8-openjdk-amd64\"\n",
    "os.environ[\"SPARK_HOME\"] = \"/content/spark-2.4.5-bin-hadoop2.7\"\n",
    "\n",
    "# Start a SparkSession\n",
    "import findspark\n",
    "findspark.init()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 224
    },
    "colab_type": "code",
    "id": "HviLso_b89tW",
    "outputId": "389d72fb-f55d-4887-a13f-6c4a23ad8eef"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "--2020-05-17 20:36:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar\n",
      "Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228\n",
      "Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.\n",
      "HTTP request sent, awaiting response... 200 OK\n",
      "Length: 914037 (893K) [application/java-archive]\n",
      "Saving to: ‘postgresql-42.2.9.jar.4’\n",
      "\n",
      "postgresql-42.2.9.j 100%[===================>] 892.61K  4.59MB/s    in 0.2s    \n",
      "\n",
      "2020-05-17 20:36:53 (4.59 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]\n",
      "\n"
     ]
    }
   ],
   "source": [
    "!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "3FP8HaLokHui"
   },
   "outputs": [],
   "source": [
    "from pyspark.sql import SparkSession\n",
    "spark = SparkSession.builder.appName(\"GamingCloudETL\").config(\"spark.driver.extraClassPath\",\"/content/postgresql-42.2.9.jar\").getOrCreate()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 224
    },
    "colab_type": "code",
    "id": "9LSw6QzzkMkY",
    "outputId": "0632f7ea-da25-4892-a680-d86c3f4dd202"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+\n",
      "|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|\n",
      "+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+\n",
      "|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|\n",
      "|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games|          5|            0|          0|   N|                Y|          Five Stars|             Awesome| 2015-08-31|\n",
      "|         US|   45765011| R3YOOS71KM5M9|B00DNHLFQA|     951665344|Command & Conquer...|Digital_Video_Games|          5|            0|          0|   N|                Y|Hail to the great...|If you are preppi...| 2015-08-31|\n",
      "|         US|     113118|R3R14UATT3OUFU|B004RMK5QG|     395682204|Playstation Plus ...|Digital_Video_Games|          5|            0|          0|   N|                Y|          Five Stars|             Perfect| 2015-08-31|\n",
      "|         US|   22151364| RV2W9SGDNQA2C|B00G9BNLQE|     640460561|Saints Row IV - E...|Digital_Video_Games|          5|            0|          0|   N|                Y|          Five Stars|            Awesome!| 2015-08-31|\n",
      "+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+\n",
      "only showing top 5 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "from pyspark import SparkFiles\n",
    "url =\"https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz\"\n",
    "spark.sparkContext.addFile(url)\n",
    "df = spark.read.csv(SparkFiles.get(\"amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz\"), sep=\"\\t\", header=True)\n",
    "\n",
    "df.show(5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 224
    },
    "colab_type": "code",
    "id": "CZNN9BrPlY-4",
    "outputId": "d299d3f4-1fd5-4901-9385-50e724400007"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+\n",
      "|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|\n",
      "+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+\n",
      "|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|\n",
      "|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games|          5|            0|          0|   N|                Y|          Five Stars|             Awesome| 2015-08-31|\n",
      "|         US|   45765011| R3YOOS71KM5M9|B00DNHLFQA|     951665344|Command & Conquer...|Digital_Video_Games|          5|            0|          0|   N|                Y|Hail to the great...|If you are preppi...| 2015-08-31|\n",
      "|         US|     113118|R3R14UATT3OUFU|B004RMK5QG|     395682204|Playstation Plus ...|Digital_Video_Games|          5|            0|          0|   N|                Y|          Five Stars|             Perfect| 2015-08-31|\n",
      "|         US|   22151364| RV2W9SGDNQA2C|B00G9BNLQE|     640460561|Saints Row IV - E...|Digital_Video_Games|          5|            0|          0|   N|                Y|          Five Stars|            Awesome!| 2015-08-31|\n",
      "+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+\n",
      "only showing top 5 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df = df.dropna()\n",
    "df.show(5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 34
    },
    "colab_type": "code",
    "id": "juyyGTn4SeaP",
    "outputId": "a802e3ce-e55c-4b8c-f064-2c1aad905a52"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "145425"
      ]
     },
     "execution_count": 6,
     "metadata": {
      "tags": []
     },
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df.count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 306
    },
    "colab_type": "code",
    "id": "8DSJ28qCShIb",
    "outputId": "7aaf5cd6-b63e-4d2a-b17d-b05cbeaa03be"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "root\n",
      " |-- marketplace: string (nullable = true)\n",
      " |-- customer_id: string (nullable = true)\n",
      " |-- review_id: string (nullable = true)\n",
      " |-- product_id: string (nullable = true)\n",
      " |-- product_parent: string (nullable = true)\n",
      " |-- product_title: string (nullable = true)\n",
      " |-- product_category: string (nullable = true)\n",
      " |-- star_rating: string (nullable = true)\n",
      " |-- helpful_votes: string (nullable = true)\n",
      " |-- total_votes: string (nullable = true)\n",
      " |-- vine: string (nullable = true)\n",
      " |-- verified_purchase: string (nullable = true)\n",
      " |-- review_headline: string (nullable = true)\n",
      " |-- review_body: string (nullable = true)\n",
      " |-- review_date: string (nullable = true)\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df.printSchema()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "colab_type": "text",
    "id": "IifpzN6zkN5Q"
   },
   "source": [
    "## Transform DataFrame to fit review_id table"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 204
    },
    "colab_type": "code",
    "id": "E72hLCa7kT_D",
    "outputId": "9e301a24-224a-4ee8-fcb1-e01a1d487159"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+--------------+-----------+----------+--------------+-----------+\n",
      "|     review_id|customer_id|product_id|product_parent|review_date|\n",
      "+--------------+-----------+----------+--------------+-----------+\n",
      "| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|\n",
      "|R1WFOQ3N9BO65I|     133437|B00F4CEHNK|     341969535| 2015-08-31|\n",
      "| R3YOOS71KM5M9|   45765011|B00DNHLFQA|     951665344| 2015-08-31|\n",
      "|R3R14UATT3OUFU|     113118|B004RMK5QG|     395682204| 2015-08-31|\n",
      "| RV2W9SGDNQA2C|   22151364|B00G9BNLQE|     640460561| 2015-08-31|\n",
      "+--------------+-----------+----------+--------------+-----------+\n",
      "only showing top 5 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "review_id_df = df.select([\"review_id\", \"customer_id\", \"product_id\", \"product_parent\", \"review_date\"])\n",
    "review_id_df.show(5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "NO3ME1MIS651"
   },
   "outputs": [],
   "source": [
    "review_id_df = review_id_df.dropDuplicates([\"review_id\"])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 136
    },
    "colab_type": "code",
    "id": "Ktr6NZQaS_IP",
    "outputId": "8feaaee6-95c3-43ad-ab63-8188bacbcffa"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "root\n",
      " |-- review_id: string (nullable = true)\n",
      " |-- customer_id: integer (nullable = true)\n",
      " |-- product_id: string (nullable = true)\n",
      " |-- product_parent: integer (nullable = true)\n",
      " |-- review_date: date (nullable = true)\n",
      "\n"
     ]
    }
   ],
   "source": [
    "from pyspark.sql.types import * \n",
    "\n",
    "review_id_df = review_id_df.withColumn(\"customer_id\",review_id_df[\"customer_id\"].cast(IntegerType()))\\\n",
    "    .withColumn(\"product_parent\",review_id_df[\"product_parent\"].cast(IntegerType()))\\\n",
    "    .withColumn(\"review_date\",review_id_df[\"review_date\"].cast(DateType()))\n",
    "\n",
    "review_id_df.printSchema()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "colab_type": "text",
    "id": "_UaiBi_bpIoM"
   },
   "source": [
    "## Transform DataFrame to fit products table"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 204
    },
    "colab_type": "code",
    "id": "7y_FeNjfo6kG",
    "outputId": "5f8ad8f7-6c96-4885-bf8f-dfe63d059014"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+----------+--------------------+\n",
      "|product_id|       product_title|\n",
      "+----------+--------------------+\n",
      "|B013PURRZW|Madden NFL 16 - X...|\n",
      "|B00F4CEHNK| Xbox Live Gift Card|\n",
      "|B00DNHLFQA|Command & Conquer...|\n",
      "|B004RMK5QG|Playstation Plus ...|\n",
      "|B00G9BNLQE|Saints Row IV - E...|\n",
      "+----------+--------------------+\n",
      "only showing top 5 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "products_df = df.select([\"product_id\", \"product_title\"])\n",
    "products_df.show(5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "Xu4a4S4RTHCX"
   },
   "outputs": [],
   "source": [
    "products_df = products_df.dropDuplicates([\"product_id\"])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 85
    },
    "colab_type": "code",
    "id": "HrETIlCPTI3P",
    "outputId": "f9f76c1e-1f95-482d-da5c-9acade030cd3"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "root\n",
      " |-- product_id: string (nullable = true)\n",
      " |-- product_title: string (nullable = true)\n",
      "\n"
     ]
    }
   ],
   "source": [
    "products_df.printSchema()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "colab_type": "text",
    "id": "ylBK2P4BqgHS"
   },
   "source": [
    "## Transform DataFrame to fit customers table"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 204
    },
    "colab_type": "code",
    "id": "0Ww33VqXqsfq",
    "outputId": "3d2dc945-f793-4585-ef91-92a8cf4c70c2"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----------+\n",
      "|customer_id|\n",
      "+-----------+\n",
      "|   21269168|\n",
      "|     133437|\n",
      "|   45765011|\n",
      "|     113118|\n",
      "|   22151364|\n",
      "+-----------+\n",
      "only showing top 5 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "customers_df = df.select([\"customer_id\"])\n",
    "customers_df.show(5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 204
    },
    "colab_type": "code",
    "id": "xwKI97AUTSOE",
    "outputId": "74851c43-a495-4436-ed4b-5664053f387e"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----------+-----+\n",
      "|customer_id|count|\n",
      "+-----------+-----+\n",
      "|   40550841|    1|\n",
      "|    1136652|    1|\n",
      "|   36526525|    1|\n",
      "|     564588|    1|\n",
      "|    2830122|    1|\n",
      "+-----------+-----+\n",
      "only showing top 5 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "customers_df = customers_df.groupBy(\"customer_id\").count()\n",
    "customers_df.orderBy(\"customer_id\").select([\"customer_id\", \"count\"])\n",
    "customers_df.show(5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 204
    },
    "colab_type": "code",
    "id": "9JMQuaNiTWAi",
    "outputId": "9981ce3f-5444-4982-dd25-a6d995650d78"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----------+--------------+\n",
      "|customer_id|customer_count|\n",
      "+-----------+--------------+\n",
      "|   40550841|             1|\n",
      "|    1136652|             1|\n",
      "|   36526525|             1|\n",
      "|     564588|             1|\n",
      "|    2830122|             1|\n",
      "+-----------+--------------+\n",
      "only showing top 5 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "customers_df = customers_df.withColumnRenamed(\"count\", \"customer_count\")\n",
    "customers_df.show(5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 85
    },
    "colab_type": "code",
    "id": "zeOeFTK8TZL5",
    "outputId": "cf3864d8-dde3-4bd5-ace5-446d65526ba0"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "root\n",
      " |-- customer_id: integer (nullable = true)\n",
      " |-- customer_count: integer (nullable = false)\n",
      "\n"
     ]
    }
   ],
   "source": [
    "customers_df = customers_df.withColumn(\"customer_id\",customers_df[\"customer_id\"].cast(IntegerType()))\\\n",
    "    .withColumn(\"customer_count\",customers_df[\"customer_count\"].cast(IntegerType()))\n",
    "\n",
    "customers_df.printSchema()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "colab_type": "text",
    "id": "0MJxZpf11QI5"
   },
   "source": [
    "## Transform DataFrame to fit vine table"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 204
    },
    "colab_type": "code",
    "id": "9hWzVw8U1QhQ",
    "outputId": "e68789a5-4803-4ee1-954c-21708972d3bf"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+--------------+-----------+-------------+-----------+----+\n",
      "|     review_id|star_rating|helpful_votes|total_votes|vine|\n",
      "+--------------+-----------+-------------+-----------+----+\n",
      "| RSH1OZ87OYK92|          2|            2|          3|   N|\n",
      "|R1WFOQ3N9BO65I|          5|            0|          0|   N|\n",
      "| R3YOOS71KM5M9|          5|            0|          0|   N|\n",
      "|R3R14UATT3OUFU|          5|            0|          0|   N|\n",
      "| RV2W9SGDNQA2C|          5|            0|          0|   N|\n",
      "+--------------+-----------+-------------+-----------+----+\n",
      "only showing top 5 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "vine_df = df.select([\"review_id\",\"star_rating\", \"helpful_votes\", \"total_votes\", \"vine\"])\n",
    "vine_df.show(5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "NoWows9oTeZv"
   },
   "outputs": [],
   "source": [
    "vine_df = vine_df.dropDuplicates([\"review_id\"])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 136
    },
    "colab_type": "code",
    "id": "V0s7tgvDTe3Z",
    "outputId": "8044523b-0219-4437-9ccf-419737680515"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "root\n",
      " |-- review_id: string (nullable = true)\n",
      " |-- star_rating: integer (nullable = true)\n",
      " |-- helpful_votes: integer (nullable = true)\n",
      " |-- total_votes: integer (nullable = true)\n",
      " |-- vine: string (nullable = true)\n",
      "\n"
     ]
    }
   ],
   "source": [
    "vine_df = vine_df.withColumn(\"star_rating\",vine_df[\"star_rating\"].cast(IntegerType()))\\\n",
    "    .withColumn(\"helpful_votes\",vine_df[\"helpful_votes\"].cast(IntegerType()))\\\n",
    "    .withColumn(\"total_votes\",vine_df[\"total_votes\"].cast(IntegerType()))\n",
    "\n",
    "vine_df.printSchema()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "colab_type": "text",
    "id": "-n5eJqaB3ZY9"
   },
   "source": [
    "## Write DataFrames to Database"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "uXQe3s_ElN2U"
   },
   "outputs": [],
   "source": [
    "mode = \"append\"\n",
    "jdbc_url=\"jdbc:postgresql://mypostgresdb.cww47e97r82y.us-east-2.rds.amazonaws.com:5432/my_data_class_db\"\n",
    "config = {\"user\":\"root\", \n",
    "          \"password\": \"\", \n",
    "          \"driver\":\"org.postgresql.Driver\"}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "cWcrjAYD0UHN"
   },
   "outputs": [],
   "source": [
    "review_id_df.write.jdbc(url=jdbc_url, table=\"review_id_table\", mode=mode, properties=config)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "RQwA1SDM0U7F"
   },
   "outputs": [],
   "source": [
    "products_df.write.jdbc(url=jdbc_url, table=\"products\", mode=mode, properties=config)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "71K6d1b60UQ1"
   },
   "outputs": [],
   "source": [
    "customers_df.write.jdbc(url=jdbc_url, table=\"customers\", mode=mode, properties=config)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "colab": {},
    "colab_type": "code",
    "id": "25_fsOV51wZP"
   },
   "outputs": [],
   "source": [
    "vine_df.write.jdbc(url=jdbc_url, table=\"vine_table\", mode=mode, properties=config)\n"
   ]
  }
 ],
 "metadata": {
  "colab": {
   "collapsed_sections": [],
   "name": "cloud_etl_gaming_analysis.ipynb",
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 1
}